<a href="https://colab.research.google.com/github/DnJAppliedDL/Machine_Learning_Tutorial/blob/main/PyTorch_Lightning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#【機器學習系列】利用PyTorch-Lightning來簡化開發流程

<div class="text-center">
    <img src="https://cdn-images-1.medium.com/max/800/1*IMGOKBIN8qkOBt5CH55NSw.png">
</div>


## 前言

作為與TensorFlow同等熱門的機器學習框架，PyTorch以自由度高以及可控程度高為特點，提供使用者高度的可自定義訓練流程。但在高度的自由之下，卻也存在著一些不方便的地方。例如需要自己呼叫 `loss.backward()` 、 `optimizer.step()` 以及 `optimizer.zero_grad()` 等函式來進行梯度的計算等操作。這些工程碼的順序在一定程度上並不會影響訓練的進行，但沒有編寫到這些工程碼卻會造成訓練的失敗。Pytorch-Lightning就是一個以減少使用者發生這類錯誤，並提升使用者編寫訓練流程的效率為目標開發的套件。

## PyTorch回顧

在介紹PyTorch-Lightning的使用前，我們可以先來回顧一下在PyTorch中要如何編寫一個以多層感知器為模型，並進行手寫數字照片的圖像辨識任務。以下是一個範例程式：

In [ ]:
# Import libraries
import torchvision 
import torch 
import numpy as np
import pandas as pd

# Transform
# Step 1: Convert to Tensor
# Step 2: Normalize with mean = 0.5 and std= 0.5
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,), (0.5,)),]
)

# Obtain MNIST training dataset.
# Transform is a custom functon define by user, 
# which allow user to apply the transformation when loading dataset.
mnist_train = torchvision.datasets.MNIST(
  root='MNIST', 
  download=True, 
  train=True, 
  transform=transform
)
                            
# Obtain MNIST test dataset
mnist_test = torchvision.datasets.MNIST(
  root='MNIST', 
  download=True, 
  train=False, 
  transform=transform
)

# Build train data loader
trainLoader = torch.utils.data.DataLoader(mnist_train, 
                                          batch_size=64, 
                                          shuffle=True, 
                                          pin_memory=True, 
                                          num_workers=4)

# Build test data loader
testLoader = torch.utils.data.DataLoader(mnist_test, 
                                         batch_size=64, 
                                         shuffle=False, 
                                         pin_memory=True, 
                                         num_workers=4)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = torch.nn.Sequential(
      torch.nn.Linear(in_features=784, out_features=128),
      torch.nn.ReLU(),
      torch.nn.Linear(in_features=128, out_features=64),
      torch.nn.ReLU(),
      torch.nn.Linear(in_features=64, out_features=10),
      torch.nn.LogSoftmax(dim=1)
).to(device)

EPOCHS = 10 
LR = 1e-2 
OPTIMIZER = 'adam' 

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    # 定義變數用以儲存單次訓練週期中的loss以及acc值
    running_loss = list()
    running_acc = 0.0

    for times, data in enumerate(trainLoader):

        # 宣告模型訓練狀態
        model.train()

        # 取得資料，並將其傳遞至相應裝置
        inputs, labels = data[0].to(device), data[1].to(device)

        # 將影像維度改為第一維度長度為inputs.shape[0]，第二維度為所剩維度展平的長度
        inputs = inputs.view(inputs.shape[0], -1)

        # 將既存梯度歸零
        optimizer.zero_grad()

        # 將資料導入模型，並取得模型輸出
        outputs = model(inputs)

        # 將模型輸出轉為整數
        predicted = torch.max(outputs.data, 1)[1]
        
        # 利用損失函數計算loss
        loss = criterion(outputs, labels)

        # 進行反向傳播
        loss.backward()

        # 更新參數
        optimizer.step()

        # 紀錄週期內的損失值
        running_loss.append(loss.item())

        # 計算週期內正確預測的數量
        running_acc += (labels==predicted).sum().item()
    
    # 計算週期為單位的loss以及acc
    _epoch_loss = torch.tensor(running_loss).mean()
    _epoch_acc = running_acc/(len(trainLoader)*64)

    # 輸出資訊
    print(f"Epoch : {epoch+1}, Epoch loss: {_epoch_loss:.4f}, Epoch Acc: {_epoch_acc:.2f}")
print('Training Finished.')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/MNIST/raw/train-images-idx3-ubyte.gz to MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/MNIST/raw

Epoch : 1, Epoch loss: 0.3682, Epoch Acc: 0.89
Epoch : 2, Epoch loss: 0.2406, Epoch Acc: 0.93
Epoch : 3, Epoch loss: 0.2313, Epoch Acc: 0.93
Epoch : 4, Epoch loss: 0.2091, Epoch Acc: 0.94
Epoch : 5, Epoch loss: 0.2095, Epoch Acc: 0.94
Epoch : 6, Epoch loss: 0.2063, Epoch Acc: 0.94
Epoch : 7, Epoch loss: 0.1929, Epoch Acc: 0.95
Epoch : 8, Epoch loss: 0.1950, Epoch Acc: 0.95
Epoch : 9, Epoch loss: 0.1872, Epoch Acc: 0.95
Epoch : 10, Epoch loss: 0.1778, Epoch Acc: 0.95
Training Finished.


在原生的PyTorch中（PyTorch-Lightning稱其為Vanilla PyTorch），建立模型有以下兩種方法：
以繼承 `nn.module` 的子類來編寫模型，並利用類別的 `forward()` 來定義如何將輸入數據送到模型的每一層進行計算。
以 `torch.nn.Sequential` 函式來建立一個序列模型。

在建立的模型之後，我們需要使用一個 for 迴圈來建立訓練迴圈，並依次從建立好的 `DataLoader` 物件中取出資料，並送入模型，再取得模型的結果。將取得的結果以損失函數計算損失值後，進行反向傳播。倘若有需要紀錄正確值以及相關衡量的標準，我們也需要自行建構計算的函式。
在原生的PyTorch中，還有一個非常麻煩的部份―-變數的移動。倘若使用GPU進行訓練，我們需要手動利用 `to()` 來將模型以及變數送到GPU中，並利用 `detach()` 來將參數移動回到CPU。無論是先前提過的工程碼，以及變數的移動，都是繁瑣、不必要、且容易造成問題的部份。在PyTorch-Lightning的協助下，我們可以免去編寫這些工程碼的必要，並透過其所提供的功能來大幅簡化整個訓練流程的設計。

## 開始使用PyTorch-Lightning

### 安裝PyTorch-Lightning

**PyTorch-Lightning**可以透過 pip 來進行安裝。

In [ ]:
!pip -q install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.5 MB/s eta 0:00:00


### 使用PyTorch-Lightning

在`Pytorch-Lightning`的使用上，我們需要透過繼承 `LightningModule` 來建立一個子類，並透過在子類中建構 `training_ste` 、 `validation_step`以及 `test_step` 來宣告我們的訓練、驗證、以及測試流程，並利用建構 `forward()` 來定義資料的流向。在這個子類中，我們不需要自己呼叫 `loss.backward()`、 `optimizer.step()` 以及 `optimizer.zero_grad()` 等函式也不需要透過 `to()` 來定義資料該如何被送到指定裝置。我們只需要在這個子類中編寫好資料要從哪裡來，要到哪裡去，並且使用 `configure_optimizer()` 來定義我們要用的`Optimizer`即可。

### 建立類別

首先，我們建立一個繼承 `LightningModule` 的類別，名為 `MyModule` ，並建立一個全連結層 `self.l1` ，並使用`PyTorch-Lightning`所提供的`torchmetrics.Accuracy`函式來計算準確率。

```
import torch
import pytorch_lightning as pl
from torch.nn import functional as F
from torchmetrics import Accuracy

class MyModule(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = torch.nn.Linear(28 * 28, 10)
    self.accuracy = torchmetrics.Accuracy()
```
接下來，我們需要建構 `forward()` 函式來設計資料在模型結構內的流向，並透過建構 `training_step()` 來定義訓練流程。
```
class MyModule(pl.LightningModule):
  # skip...
  def forward(self, x):
    out = self.l1(x.view(x.size(0), -1))
    return torch.relu(out)
  def training_step(self, batch, batch_nb):
    x, y = batch
    out = self(x)
    self.accuracy(out, y)
    loss = F.cross_entropy(self(x), y)
    self.log("train_loss", loss, on_epoch=True)
    self.log("train_acc", self.accuracy, on_epoch=True)
    return loss
```
最後，我們透過建構 `configure_optimizer()` 來設定我們想用的`Optimizer`。
```
class MyModule(pl.LightningModule):
  # skip...
  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=0.02)
```
接下來，我們就可以透過 `pl.Trainer` 搭配相關的超參數，建立一個 `trainer` 物件，並進行訓練。

### 建立Trainer物件
接下來，我們將建立一個`Trainer`物件，並設定訓練10個週期，以及使用GPU進行訓練。
```
trainer = pl.Trainer(max_epochs=10, devices=1, accelerator="gpu")
```


### 開始訓練
接下來，我們將開始進行訓練。資料方面，我們將使用官方提供的MNIST手寫數字影像資料集作為輸入資料。

```
import os
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

BATCH_SIZE = 256

dataset = MNIST(
  os.getcwd(),
  train=True,
  download=True,
  transform=transform.ToTensor()
)

train, val = random_split(dataset, [55000, 5000])

train_ds = DataLoader(
  train,
  num_workers=4,
  pin_memory=True,
  batch_size=BATCH_SIZE
)

val_ds = DataLoader(
  val,
  num_workers=4,
  pin_memory=True,
  batch_size=BATCH_SIZE
)

mnist_model = MyModule()

trainer.fit(
  mnist_model,
  train_ds,
  val_ds,
)
```

## 完整PyTorch-Lightning程式碼

讀者可以嘗試執行以下程式碼來實際觀察PyTorch-Lightning的運作。

In [ ]:
import torch
import os
import pytorch_lightning as pl
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchmetrics import Accuracy

class MyModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
    def forward(self, x):
        out = self.l1(x.view(x.size(0), -1))
        return torch.relu(out)
    def training_step(self, batch, batch_nb):
        x, y = batch
        out = self(x)
        self.accuracy(out, y)
        loss = F.cross_entropy(self(x), y)
        self.log("train_loss", loss, on_epoch=True)
        self.log("train_acc", self.accuracy, on_epoch=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

trainer = pl.Trainer(max_epochs=10, devices=1, accelerator="gpu")

BATCH_SIZE = 256

dataset = MNIST(
  os.getcwd(),
  train=True,
  download=True,
  transform=transforms.ToTensor()
)

train, val = random_split(dataset, [55000, 5000])

train_ds = DataLoader(
  train,
  num_workers=4,
  pin_memory=True,
  batch_size=BATCH_SIZE
)

val_ds = DataLoader(
  val,
  num_workers=4,
  pin_memory=True,
  batch_size=BATCH_SIZE
)

mnist_model = MyModule()

trainer.fit(
  mnist_model,
  train_ds,
  val_ds,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/configuration_validator.py:106: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | l1       | Linear             | 7.9 K 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
7.9 K     Trainable pa

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
